# Load libraries and Themes

Run in user_R

In [3]:
suppressPackageStartupMessages({
    suppressWarnings({library(Seurat)
        library(SeuratObject)
        library(SeuratDisk)
        library(ggplot2)
        library(tidyverse)
        library(harmony)
        library(DESeq2)
        library(EnhancedVolcano)
        library(Rsamtools)
        library(svglite)
        library(viridis)
        library(gridExtra)
        library(scCustomize)
        library(ggpubr)
        library(pals)
        library(patchwork)
        library(ComplexUpset)
    })})

In [ ]:
#Color Palettes

palette.doublets <- c(
    '#197278',#turqu
    '#C1C1C1'#grey
)

palette.2 <- c(
    "#990902",
    "#0303a3"
)

palette.4 <- DiscretePalette(4, palette = "stepped", shuffle = TRUE)

palette.12 <- DiscretePalette(12, palette = "stepped", shuffle = TRUE)

palette.15 <- DiscretePalette(12, palette = "stepped", shuffle = TRUE)

palette <- c(
    "#990902",#CM_0
    "#0303a3",#EC-end
    "#ebe6c7",#FB
    "#077a01",#MΦ-M2
    "#05b1eb",#EC-cap
    "#660202",#PER
    "#4b048a",#MESO
    "#7d0701",#CM_1
    "#0FB602",#TC
    "#0324fc",#EC-lym
    "#FFD100",#SC
    "#d90f04",#CM_2
    "#aad902"#BC
)

palette.CM <- c(
    "#AF0000",#CM
    "#0303a3",#EC-end
    "#ebe6c7",#FB
    "#077a01",#MΦ-M2
    "#05b1eb",#EC-cap
    "#660202",#PER
    "#4b048a",#MESO
    "#0FB602",#TC
    "#0324fc",#EC-lym
    "#FFD100",#SC
    "#aad902"#BC
)
 
palette.treatment <- c(
    '#AF0000',#ALDO
    '#C1C1C1'#CTRL
)

In [ ]:
umap_theme <- theme(
  axis.line=element_blank(),
  axis.text.x=element_blank(),
  axis.text.y=element_blank(),
  axis.ticks=element_blank(),
  axis.title.x=element_blank(),
  axis.title.y=element_blank(),
  panel.background=element_blank(),
  panel.border=element_blank(),
  panel.grid.major=element_blank(),
  panel.grid.minor=element_blank()
)

vln_theme_1 <- theme(
  axis.text.x = element_blank(),
  axis.text.y = element_blank(),
  panel.background = element_blank(),
  axis.title.x = element_blank(),
  axis.title.y = element_blank(),
  panel.border = element_blank(),
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  text = element_text(size = 25)
)

vln_theme_2 <- theme(
  panel.background = element_blank(),
  panel.border = element_blank(),
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
)

In [ ]:
setwd("/media/daten/dmeral/scseq_analysis/2024_LA_CTRL_ALDO")

In [ ]:
set.seed(1234)

# UMAPs

In [ ]:
# Load after subcluster analysis is finished 
obj.subcluster_CM <- LoadH5Seurat("subcluster/Subcluster_CM_down.h5seurat")

In [ ]:
# Subset to CM clusters
sub_CM <- subset(x = obj.subcluster_CM, subset = seurat_clusters %in% c("0", "1"))

In [ ]:
# Subset the `sub_CM` object to include only cells with detectable Fkbb5 expression
sub_CM_expressionsub <- subset(x = sub_CM, subset = Nr3c2 > 0)

In [ ]:
# Load full seurat obj
obj.full <- LoadH5Seurat("seurat_objects/obj_seu_merge_harmony_sgl_addmodule_downsample_rename.h5seurat")

In [ ]:
# Number of nuclei (or cells) in the Seurat object
num_nuclei <- ncol(obj.full)
cat("Number of nuclei:", num_nuclei, "\n")

# Median number of genes per nucleus
# This uses the 'nFeature_RNA' column in the metadata, which stores the number of genes detected per cell
median_genes <- median(obj.full@meta.data$nFeature_RNA)
cat("Median number of genes per nucleus:", median_genes, "\n")


In [ ]:
sub_full <- subset(x = obj.full, subset = cell_type %in% c("CM_0", "MΦ", "FB", "PER", "EC-end", "EC-cap", "CM_2", "CM_1", "MESO"))

In [ ]:
# Split the Seurat object by treatment
seurat_list_full <- SplitObject(obj.full, split.by = "treatment")

## MR expression

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 300)

# Generate the UMAP plots for Zbtb16 with split by treatment
UMAP_MR <- FeaturePlot(obj.full, features = "Nr3c2",
                                pt.size = 0.1, order = TRUE, cols = viridis(258)) + umap_theme



# Save the plot
ggsave("Plots/UMAP_MR.svg", UMAP_MR, units = "cm", dpi = 300, width = 15, height = 10)

UMAP_MR

In [ ]:
# Combine CM clusters 
cluster_annotations_CMcomb <- list(
    "0" = "CM",
    "1" = "EC-end",
    "2" = "FB",
    "3" = "MΦ",
    "4" = "EC-cap",
    "5" = "PER",
    "6" = "MESO",
    "7" = "CM",
    "8" = "TC",
    "9" = "EC-lym",
    "10" = "SC",
    "11" = "CM",
    "12" = "BC")

In [ ]:
names(cluster_annotations_CMcomb) <- levels(obj.full)
obj.full <- RenameIdents(obj.full, cluster_annotations_CMcomb)

obj.full$"cell_type_CMcomb" <- Idents(obj.full)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 300)

DimPlot(obj.full, label = TRUE)

In [ ]:
options(repr.plot.width = 3, repr.plot.height = 4, repr.plot.res = 300)

# Create DotPlot for each treatment
DOT_MR <- DotPlot(
  obj.full,
  features = c("Nr3c2")) + 
  geom_point(aes(size = pct.exp), shape = 21, colour = "black", stroke = 0.5) +  
  guides(
    size = guide_legend(override.aes = list(shape = 21, colour = "black", fill = "white")),  
    colour = guide_colorbar(frame.colour = "black", ticks.colour = "black")  
  ) +
  theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
  theme(panel.border = element_rect(color = "black", linewidth = 0.5)) +
  scale_colour_distiller(palette = "OrRd", direction = 0) +
  scale_y_discrete(limits = rev) +
  labs(x = NULL, y = NULL) +  # Removes axis labels
  ggtitle("MR expression")

# Suppress warnings and print the combined plot
DOT_MR

# Save the combined plot to a file
ggsave("Plots/DOT_MR_expression.svg", DOT_MR, units = "cm", width = 7.5, height = 10, dpi = 300)

In [ ]:
# Define the gene of interest
gene_of_interest <- "Nr3c2"  # Adjust if using a different gene name

# Extract expression data
expr_data <- FetchData(obj.full, vars = gene_of_interest)

# Add metadata (cell types)
expr_data$cell_type_CMcomb <- obj.full$cell_type_CMcomb

# Subset to T cells only
tcell_expr <- expr_data %>% filter(cell_type_CMcomb == "TC")

# Check summary statistics
summary(tcell_expr[, gene_of_interest])


In [ ]:
# Non zero MR expresson TC
obj_Tcells <- subset(obj.full, cell_type_CMcomb == "TC")

# Extract MR expression values for T cells
mr_expr_TC <- FetchData(obj_Tcells, vars = "Nr3c2")

# Count how many T cells have nonzero MR expression
num_mr_positive_TC <- sum(mr_expr_TC$Nr3c2 > 0)

# Total number of T cells
total_Tcells <- nrow(mr_expr_TC)

# Percentage of MR+ T cells
percent_mr_positive_TC <- (num_mr_positive_TC / total_Tcells) * 100

# Print results
num_mr_positive_TC
percent_mr_positive_TC

In [ ]:
VlnPlot(obj.full, features = "Nr3c2", group.by = "cell_type_CMcomb", pt.size = FALSE, cols = palette.CM) + 
  ggtitle("MR (Nr3c2) Expression")

## CM subcluster

In [ ]:
sub_CM <- subset(x = obj.subcluster_CM, subset = seurat_clusters %in% c("0", "1"))

In [ ]:
palette.treatment

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 5, repr.plot.res = 300) 

VlnGOI <- VlnPlot(sub_CM, group.by = "cell_type_sub", 
                      features = c("Nppb", "Ctgf", "Zbtb16", "Hlf"), pt.size = 0, raster = TRUE, ncol = 4, cols = rev(palette.treatment))


ggsave("subcluster/Plots/VlnPlot_NppbCtgfZbtb16Hlf.svg", VlnGOI, units = "cm", dpi = 300, width = 18, height = 7.5)

VlnGOI

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 300)

# Generate the UMAP plots for Zbtb16 with split by treatment
UMAP_Hcn4 <- FeaturePlot(obj.subcluster_CM, features = "Hcn4",
                                pt.size = 0.1, order = TRUE, cols = viridis(258)) + umap_theme



# Save the plot
ggsave("subcluster/Plots/UMAP_Hcn4.svg", UMAP_Hcn4, units = "cm", dpi = 300, width = 10, height = 8)

UMAP_Hcn4

# MR target Dot plot

In [ ]:
MR_tragets_GOI <- list(c(
"Fkbp5",
"Klf9",
"Per1",
"Per2",
"Tsc22d3",
"Clock",
"Egfr",
"Kdr",
"Zfp346",
"Xdh",
"Ace",
"Zbtb16",
"Txnip"
))

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10, repr.plot.res = 300)

cluster_annotations_sgl <- list(
    "0" = "CM",
    "1" = "EC-end",
    "2" = "FB",
    "3" = "MΦ",
    "4" = "EC-cap",
    "5" = "PER",
    "6" = "MESO",
    "9" = "CM",
    "10" = "TC",
    "11" = "EC-lym",
    "12" = "SC",
    "13" = "CM",
    "14" = "BC"
)

names(cluster_annotations_sgl) <- levels(obj.full)
obj.full <- RenameIdents(obj.full, cluster_annotations_sgl)

# Split the Seurat object by treatment
seurat_list <- SplitObject(obj.full, split.by = "treatment")

Idents(obj.full) <- obj.full@meta.data$"Ident_numerical"

# Create DotPlot for each treatment
DOT_GOI_Aldosterone <- DotPlot(
  seurat_list$ALDO,
  features = unlist(MR_tragets_GOI, use.names = FALSE)) + 
  geom_point(aes(size = pct.exp), shape = 21, colour = "black", stroke = 0.5) +  
  guides(
    size = guide_legend(override.aes = list(shape = 21, colour = "black", fill = "white")),  
    colour = guide_colorbar(frame.colour = "black", ticks.colour = "black")  
  ) +
  theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
  theme(panel.border = element_rect(color = "black", linewidth = 0.5)) +
  scale_colour_distiller(palette = "OrRd", direction = 0) +
  scale_y_discrete(limits = rev) +
  labs(x = NULL, y = NULL) +  # Removes axis labels
  ggtitle("Aldosterone")

DOT_GOI_Control <- DotPlot(
  seurat_list$CTRL,
  features = unlist(MR_tragets_GOI, use.names = FALSE)) + 
  geom_point(aes(size = pct.exp), shape = 21, colour = "black", stroke = 0.5) +  
  guides(
    size = guide_legend(override.aes = list(shape = 21, colour = "black", fill = "white")),  
    colour = guide_colorbar(frame.colour = "black", ticks.colour = "black")  
  ) +
  theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
  theme(panel.border = element_rect(color = "black", linewidth = 0.5)) +
  scale_colour_distiller(palette = "OrRd", direction = 0) +
  scale_y_discrete(limits = rev) +
  labs(x = NULL, y = NULL) +  # Removes axis labels
  ggtitle("Control")

# Combine the two plots vertically using patchwork
combined_plot <- DOT_GOI_Aldosterone / DOT_GOI_Control

# Suppress warnings and print the combined plot
suppressWarnings(print(combined_plot))

# Save the combined plot to a file
ggsave("Plots/DOT_MR_targets_GOI.svg", combined_plot, units = "cm", width = 16, height = 18, dpi = 300)

In [4]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 24.04.1 LTS

Matrix products: default
BLAS/LAPACK: /media/daten/dmeral/micromamba/envs/user_R/lib/libopenblasp-r0.3.26.so

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] ComplexUpset_1.3.3          UpSetR_1.4.0               
 [3] patchwork_1.3.0             pals_1.8                   
 [5] ggpubr_0.6.0                scCustomize_2.1.2          
 [7] gridExtra_2.3               viridis_0.6.5              
 [9] viridisLite_0.4.2           svglite_2.1.3              
[11] Rsamtools_2.14.0            Bio